In [1]:
import os
import os.path
import shutil
import math
import subprocess
import numpy
import pickle
import pandas as pd
import matplotlib.pyplot as plt


from amuse.units import units, constants, nbody_system
from amuse.units.core import enumeration_unit
from amuse.units.generic_unit_converter import ConvertBetweenGenericAndSiUnits
from amuse.datamodel import Particles, Particle, ParticlesSuperset
from amuse.io import write_set_to_file, read_set_from_file
from amuse.support.exceptions import AmuseException

from amuse.datamodel.particle_attributes import total_angular_momentum, kinetic_energy
from amuse.ext.orbital_elements import orbital_elements_from_binary, orbital_period_to_semimajor_axis, get_orbital_elements_from_binaries
from amuse.ext.star_to_sph import convert_stellar_model_to_SPH, StellarModelInSPH
from amuse.ext.sink import new_sink_particles
from amuse.ext.star_to_sph import convert_stellar_model_to_SPH, StellarModelInSPH
from amuse.couple.bridge import Bridge, CalculateFieldForParticles

from amuse.community.evtwin.interface import EVtwin
from amuse.community.mesa.interface import MESA # original
from amuse.community.gadget2.interface import Gadget2
from amuse.community.twobody.twobody import TwoBody
from amuse.community.huayno.interface import Huayno

import matplotlib
#matplotlib.use("Agg")
from matplotlib import pyplot
from amuse.plot import scatter, xlabel, ylabel, plot,loglog,semilogx,semilogy, sph_particles_plot
from amuse.plot import pynbody_column_density_plot, HAS_PYNBODY

In [2]:
def dynamical_time_scale(m, r, G=constants.G):
    return numpy.sqrt(r**3/(2*constants.G*m))

def Kelvin_Helmholtz_timescale(m, r, L):
    return constants.G*m**2/(r*L)

def nuclear_time_scale(l, L):
    return 7e-4 * m * constants.c**2/L

def relaxation_time_scale(N, M, R, G=constants.G):
    return 0.138*N/numpy.log(0.4*N) * dynamical_time_scale(M, R, G=constants.G)

def semimajor_axis(orb_per,M_tot):
    orb_per = orb_per | units.day
    M_tot = M_tot | units.MSun
    return (((constants.G * M_tot*orb_per**2)/(4 * numpy.pi**2))**(1/3)).as_quantity_in(units.AU)
    

In [26]:
def new_working_directory():
    i = 0
    current_directory = os.getcwd()
    while os.path.exists(os.path.join(current_directory, "run_{0:=03}".format(i))):
        i += 1
    new_directory = os.path.join(current_directory, "run_{0:=03}".format(i))
    os.mkdir(new_directory)
    print("Created new directory for output:", new_directory)
    os.mkdir(os.path.join(new_directory, "plots"))
    os.mkdir(os.path.join(new_directory, "snapshots"))
    shutil.copy(__file__, new_directory)
    if not os.path.exists(os.path.join(current_directory, "giant_models")):
        os.mkdir(os.path.join(current_directory, "giant_models"))
    os.chdir(new_directory)

def get_relative_velocity(total_mass, semimajor_axis, ecc):
    '''
    calculates and returns the relative velocity of a star as ( GM *((1+e)/(1-e))/alpha )^0.5
    '''
    return (constants.G * total_mass * ((1.0 + ecc)/(1.0 - ecc)) / semimajor_axis).sqrt()

def set_up_inner_binary():
    '''
    Sets the orbital parameters of the inner binary, moves the binary
    to the center of mass of the three-body system:

    particles.position -= particles.center_of_mass()
    particles.velocity -= particles.center_of_mass_velocity()

    and returns the binary
    '''
    #semimajor_axis = 0.048273275517640366 | units.AU  # 0.133256133158 
    orbital_period = 1.10 | units.day
    eccentricity = 0
    masses = [6.5, 5.9] | units.MSun #3.2,3.1
    semimajor_axis = orbital_period_to_semimajor_axis(orbital_period, masses[0], masses[1])
    
    #orbital_period = (4 * numpy.pi**2 * semimajor_axis**3 / 
    #    (constants.G * masses.sum())).sqrt().as_quantity_in(units.day)
    
    print("   Initializing inner binary")
    print("   Orbital period inner binary:", orbital_period)
    stars =  Particles(2)
    stars.mass = masses
    stars.position = [0.0, 0.0, 0.0] | units.AU
    stars.velocity = [0.0, 0.0, 0.0] | units.km / units.s
    stars[0].x = semimajor_axis
    stars[0].vy = get_relative_velocity(stars.total_mass(), semimajor_axis, eccentricity)
    stars.move_to_center()                     # more info in /src/amuse/datamodel/particle_attributes.py
    return stars

def set_up_outer_star(inner_binary_mass):
    '''
    Input: inner binary combined mass 
    
    Sets the orbital parameters of the tertiary, takes the inner binary mass to set 
    the relative velocity of the tertiary with respect to the inner binary (the tertiary
    gravitationaly 'sees' the inner binary as a star with mass = inner_binary_mass) 
    and returns the tertiary
    '''
    #outer_orbital_sep = 0.8274288819130811 | units.AU  # 1.22726535008
    orbital_period = 52.04 | units.day
    eccentricity = 0.3     #0.15
    inclination = math.radians(16.8)  # 9.0
    
    print("   Initializing outer star")
    giant = Particle()
    giant.mass = 16 | units.MSun # 5.5
    outer_orbital_sep = orbital_period_to_semimajor_axis(orbital_period, giant.mass, inner_binary_mass)
    giant.position = outer_orbital_sep * ([math.cos(inclination), 0, math.sin(inclination)] | units.none)
    giant.velocity = get_relative_velocity(giant.mass + inner_binary_mass, 
        outer_orbital_sep, eccentricity) * ([0, 1, 0] | units.none)
    print("   Initializing outer star")
    print("   Orbital period outer binary:", orbital_period.value_in(units.day))
    return giant

def set_up_initial_conditions():
    '''
    Sets up the initial conditions for the triple system by adding the tertiary to the system.
    'stars' variable contains now the three stars but 'view_on_giant' is just the tertiary.
    It also moves the system's coordinates in to the center of mass of the triple and returns the triple and
    the tertiary alone. The local 'stars' variable is the 'triple' variable in the main code.
    '''
    stars = set_up_inner_binary()
    giant = set_up_outer_star(stars.total_mass())
    view_on_giant = stars.add_particle(giant) # This is just the tertiary and not the whole system
    stars.move_to_center()
    return stars, view_on_giant

def triple_set_up_info(triple, view_on_giant):
    
    print("\n Inner orbit:")
    inner_seperation = (triple[0].position-triple[1].position).lengths().in_(units.au)
    print("Orbital Separation of inner binary stars {:.2f} AU or {:.2f} RSun".format( \
        inner_seperation.value_in(units.au), inner_seperation.value_in(units.RSun)))
    
    sm_1 = ((triple-view_on_giant).center_of_mass() - triple[0].position).lengths().in_(units.au)
    print("Semi-major axis1 {:.2f} AU or {:.2f} RSun".format(sm_1.value_in(units.au), \
                    sm_1.value_in(units.RSun)))
    sm_2 = ((triple-view_on_giant).center_of_mass() - triple[1].position).lengths().in_(units.au)
    print("Semi-major axis1 {:.2f} AU or {:.2f} RSun".format(sm_2.value_in(units.au), \
                    sm_2.value_in(units.RSun)))
    
    print("\n Outer orbit:")
    a_bin = ((triple-view_on_giant).center_of_mass() - triple.center_of_mass()).length().in_(units.au)
    print("\nBinary system semi-major axis is {:.2f} AU or {:.2f} RSun".format(a_bin.value_in(units.au), \
                                                                               a_bin.value_in(units.RSun))) 
    a_giant = triple[2].position.length().in_(units.AU)      
    print("Giant's semi-major is axis {:0.2f} AU or {:.2f} RSun".format(a_giant.value_in(units.au), \
                                                                        a_giant.value_in(units.RSun)))
    dist_giant = (view_on_giant.position - (triple-view_on_giant).center_of_mass()).length().in_(units.au)
    print("Giant's distance from binary's COM is {:.2f} AU or {:.2f} RSun".format(dist_giant.value_in(units.au), \
                                                                                  dist_giant.value_in(units.RSun)))

def estimate_roche_radius(triple, view_on_giant):
    '''
    Estimates and returns the Roche radius of the tertiary. In the calculation of the semi-major axis of the 
    tertiary the inclination of the tertiary's orbit is also taken into account by the position
    of the tertiary in the 3D space, but the the inner binay is 'seen' as a mass point by the
    tertiary. Furthermore, it is assumed that the tertiary's orbit is circular in the calculation
    of the Roche radius. 
    
    --> At this point a correction can be made to include the eccenticity of the tertiary, in the calculation
    of the Roche lobe 
    '''
    # 'mass ratio' of giant to inner binary
    q = (view_on_giant.mass / (triple-view_on_giant).total_mass())
    # Assume ~ circular orbit:
    a = (view_on_giant.position - (triple-view_on_giant).center_of_mass()).length()
    q13 = q**(1./3.)
    q23 = q13**2
    return (a*(0.49*q23/(0.6*q23+math.log(1+q13)))).as_quantity_in(units.RSun)

def estimate_roche_radius_bin(star1, star2):
    '''
    Estimates and returns the Roche radius of the binary components. The binary orbit is circular in 
    the calculation. 
    '''
    # 'mass ratio' of giant to inner binary
    q = (star1.mass / star2.mass)
    # Assume ~ circular orbit:
    a = (star1.position - star2.position).length()
    q13 = q**(1./3.)
    q23 = q13**2
    return (a*(0.49*q23/(0.6*q23+math.log(1+q13)))).as_quantity_in(units.RSun)

def evolve_stars(triple, view_on_giant, stellar_code,radius_factor):
    se_giant = stellar_code.particles.add_particle(view_on_giant)
    #print(stellar_code.parameters)
    
    initial_mass = stellar_code.particles.mass
    initial_radius = stellar_code.particles.radius
    initial_luminosity = stellar_code.particles.luminosity
    
    stop_radius = radius_factor * estimate_roche_radius(triple, view_on_giant)
    print('Roche Lobe radius of the tertiary ={:.2f} au'.format(stop_radius.value_in(units.au)))
    #stellar_code_options = dict(redirection='file', redirect_file='stellar_evolution_code_out.log')
    
    columns_names = ['star_age', 'star_mass', 'log_R', 'log_L', 'log_Teff', 'wind(MSun/yr)', 'log_cntr_Rho', 'log_cntr_T', 'stellar type']
    mesa_output = pd.DataFrame(columns = columns_names)
    
    
    while se_giant.radius < stop_radius.in_(units.RSun):
        stellar_code.evolve_model(keep_synchronous = False)#, **stellar_code_options)
        print(se_giant.mass.in_(units.MSun), se_giant.wind.in_(units.MSun / units.yr), se_giant.age.in_(units.Myr),
             se_giant.radius.in_(units.RSun), se_giant.stellar_type)
        
        mesa_output = pd.concat([mesa_output,pd.DataFrame({'star_age':se_giant.age.value_in(units.Myr), 'star_mass': se_giant.mass.value_in(units.MSun),
                                          'log_R': numpy.log10(se_giant.radius.value_in(units.RSun)),'log_L': numpy.log10(se_giant.luminosity.value_in(units.LSun)),
                                          'log_Teff': numpy.log10(se_giant.temperature.value_in(units.K)),'wind(MSun/yr)': se_giant.wind.value_in(units.MSun / units.yr),
                                          'log_cntr_Rho': numpy.log10(se_giant.central_density.value_in(units.g / units.cm**3)),'log_cntr_T': numpy.log10(se_giant.central_temperature.value_in(units.K)),
                                          'stellar type':se_giant.stellar_type},index=[0])], ignore_index=True, sort=False)

    print(stellar_code.parameters)    
    print('\n M(t=0)=', initial_mass, \
          '\n R(t=0)=', initial_radius, \
          '\n L(t=0)=', initial_luminosity, \
          
          
          '\n at t=', se_giant.age.in_(units.Myr), \
          '\n M(t) =', stellar_code.particles.mass.in_(units.MSun), \
          '\n R(t) =', stellar_code.particles.radius.in_(units.RSun), \
          '\n L(t) =', stellar_code.particles.luminosity.in_(units.LSun), \
          '\n Stellar type =', stellar_code.particles.stellar_type, \
         )
    #mesa_output.to_csv('amuse_mesa_runs/stellar_evolution_code_output_Fov_00275', sep='\t', index=False)
  
    se_binary = stellar_code.particles.add_particles(triple - view_on_giant)
    print(stellar_code.particles.time_step)
    for particle in se_binary:
        particle.evolve_for(se_giant.age)
        print(particle.age.in_(units.Myr),particle.time_step)
    return stellar_code.particles, stellar_code, mesa_output


def energy_evolution_plot(time, kinetic, potential, thermal, figname = "energy_evolution.png"):
    time.prepend(0.0 | units.day)
    pyplot.figure(figsize = (5, 5))
    plot(time, kinetic, label='K')
    plot(time, potential, label='U')
    plot(time, thermal, label='Q')
    plot(time, kinetic + potential + thermal, label='E')
    xlabel('Time')
    ylabel('Energy')
    pyplot.legend(prop={'size':"x-small"}, loc=4)
    pyplot.savefig(figname)
    pyplot.close()

def convert_giant_to_sph(view_on_se_giant, number_of_sph_particles): #pickle_file
    '''
    Converting the 1D stellar evolution model to a gas particle distribution       # more info /src/amuse/ext/star_to_sph.py

    Converts the tertiary star to a collection of SPH particles based on the output of the stellar evolution code.
    Thus, the collection of particles is characterized by the temperature, density etc profiles that characterize the evolved
    tertiary at the moment when its radius is equal with its Roche radius. It returns the the collection of the particles.
    '''
    giant_in_sph = convert_stellar_model_to_SPH(
        view_on_se_giant,                                                              # Star particle to be converted to an SPH model
        number_of_sph_particles,                                                       
        with_core_particle = True,                                                     # Model the core as a heavy, non-sph particle
        target_core_mass  = 7.0 | units.MSun,                  # 1.4 | units.MSun      # If (with_core_particle): target mass for the non-sph particle (in paper M = 2, ?)
        do_store_composition = False                                                   # If set, store the local chemical composition on each particle 
    )
    return giant_in_sph

In [27]:
def relax_in_isolation(giant_in_sph, sph_code, output_base_name, mult_factor):
    '''
    Calculates the total mass of the star as the mass of the SPH particles representing the outer layers plus
    the mass of a non-SPH particle representing the core. Calculates the total radius of the star as the position
    of the most distant SPH particle. Calculates the dynamical evolution timescale which is used to calculate
    the relaxation time scale of the system. The latter represent the time scale on which the global characteristics
    (bulk system parameters and stellar orbital elements) of the system change.
    

     --> not sure why t_end = 10*t_dyn, if t_end is the t_relax then the constant 10 seems small based on the
     t_relax = 0.138*(N/lnγN)*t_dyn, where in this code N=50000

    '''
    total_mass = giant_in_sph.gas_particles.total_mass() + giant_in_sph.core_particle.mass  
    total_radius = max(giant_in_sph.gas_particles.position.lengths_squared()).sqrt()
    #print(total_mass.in_(units.MSun),total_radius.in_(units.RSun))
    dynamical_timescale = (total_radius**3 / (2 * constants.G * total_mass)).sqrt().as_quantity_in(units.day)
    t_end = (mult_factor * dynamical_timescale).as_quantity_in(units.day) 
    #t_end = 0.138*(N/numpy.log(0.4*N)) * dynamical_time_scale
    n_steps = 100
    hydro_code_options = dict(number_of_workers=3, redirection='file', redirect_file='hydrodynamics_code_relax_out.log')

    unit_converter = ConvertBetweenGenericAndSiUnits(total_radius, total_mass, t_end)           # more info in /src/amuse/units/generic_unit_converter.py
    hydrodynamics = sph_code(unit_converter, **hydro_code_options)
    hydrodynamics.parameters.epsilon_squared = giant_in_sph.core_radius**2                     # Softening removes the singularity in the inverse-square force
    hydrodynamics.parameters.max_size_timestep = t_end
    hydrodynamics.parameters.time_max = 1.1 * t_end
    hydrodynamics.parameters.time_limit_cpu = 7.0 | units.day
    hydrodynamics.gas_particles.add_particles(giant_in_sph.gas_particles)
    hydrodynamics.dm_particles.add_particle(giant_in_sph.core_particle)
    
    potential_energies = hydrodynamics.potential_energy.as_vector_with_length(1).as_quantity_in(units.erg)
    kinetic_energies = hydrodynamics.kinetic_energy.as_vector_with_length(1).as_quantity_in(units.erg)
    thermal_energies = hydrodynamics.thermal_energy.as_vector_with_length(1).as_quantity_in(units.erg)
    
    print("Relaxing for {:.2f} ({:.0f} * dynamical timescale)".format(t_end.value_in(units.day),mult_factor))
    times = (t_end * list(range(1, n_steps+1)) / n_steps).as_quantity_in(units.day)
    for i_step, time in enumerate(times):
        hydrodynamics.evolve_model(time)
        print("   Relaxed for:", time)
        potential_energies.append(hydrodynamics.potential_energy)
        kinetic_energies.append(hydrodynamics.kinetic_energy)
        thermal_energies.append(hydrodynamics.thermal_energy)
        
    
    hydrodynamics.gas_particles.copy_values_of_attributes_to(
        ['mass', 'x','y','z', 'vx','vy','vz', 'u', 'h_smooth'], 
        giant_in_sph.gas_particles)
    giant_in_sph.core_particle.position = hydrodynamics.dm_particles[0].position
    giant_in_sph.core_particle.velocity = hydrodynamics.dm_particles[0].velocity
    hydrodynamics.stop()
    
    
#     snapshotfile_path = 'data/Mc_7/'
#     plt.figure(figsize=(5,5))
#     plt.scatter(giant_model.gas_particles.x.value_in(units.RSun) ,giant_model.gas_particles.y.value_in(units.RSun))
#     plt.scatter(giant_model.core_particle.x.value_in(units.RSun) ,giant_model.core_particle.y.value_in(units.RSun),c='red', label=r'M$_{core}$ = 3 M$_{\odot}$')
#     plt.savefig(snapshotfile_path + output_base_name, format='png')
    
    
#     snapshotfile = snapshotfile_path+output_base_name + "_gas.amuse"
#     write_set_to_file(giant_in_sph.gas_particles, snapshotfile, format='amuse')
#     shutil.copy(snapshotfile, os.path.join("..", "giant_models"))
    
#     snapshotfile = snapshotfile_path+output_base_name + "_core.amuse"
#     # temporarily store core_radius on the core particle
#     giant_in_sph.core_particle.radius = giant_in_sph.core_radius
#     write_set_to_file(giant_in_sph.core_particle.as_set(), snapshotfile, format='amuse')
#     giant_in_sph.core_particle.radius = 0.0 | units.m
#     shutil.copy(snapshotfile, os.path.join("..", "giant_models"))
    
#     energy_evolution_plot(times, kinetic_energies, potential_energies, thermal_energies, 
#         figname = snapshotfile_path+output_base_name + "_energy_evolution.png")
    
##########################################
# Phase three: Coupling hydrodynamics with gravity in the evolution model 
##########################################

def update_sinks_radii(sink_cand):
    '''
    Updates the accretion radius of the sinks based on the Roche Lobe radius
    '''
    sink_cand[0].sink_radius = estimate_roche_radius_bin(sink_cand[0],sink_cand[1])
    sink_cand[1].sink_radius = estimate_roche_radius_bin(sink_cand[1],sink_cand[0])
    #sink_cand[1].sink_radius = 0.1 | units.au
    return sink_cand

def prepare_binary_system(dynamics_code, binary_particles):
    '''
    Prepares the binary system for the N-body code by specifing the units
    for the code.
    '''
    unit_converter = nbody_system.nbody_to_si(
        binary_particles.total_mass(),
        (binary_particles[0].position - binary_particles[1].position).length())
    system = dynamics_code(unit_converter, redirection="none")
    system.particles.add_particles(binary_particles)
    return system


def prepare_giant_system(sph_code, giant_model, view_on_giant, time_unit, n_steps):
    '''
    Prepares the tertiary for the SPH code by specifing the units
    for the code.
    '''
    hydro_code_options = dict(number_of_workers=2, 
        redirection='file', redirect_file='hydrodynamics_code_out.log')
    unit_converter = ConvertBetweenGenericAndSiUnits(
        1.0 | units.RSun, 
        giant_model.gas_particles.total_mass() + giant_model.core_particle.mass, 
        time_unit)
    system = sph_code(unit_converter, **hydro_code_options)
    system.parameters.epsilon_squared = giant_model.core_radius**2
    system.parameters.max_size_timestep = time_unit / n_steps
    system.parameters.time_max = 1.1 * time_unit
    system.parameters.time_limit_cpu = 7.0 | units.day

    giant_model.gas_particles.position += view_on_giant.position
    giant_model.gas_particles.velocity += view_on_giant.velocity
    
    giant_model.core_particle.position += view_on_giant.position
    giant_model.core_particle.velocity += view_on_giant.velocity
    
    system.gas_particles.add_particles(giant_model.gas_particles)
    system.dm_particles.add_particle(giant_model.core_particle)
    return system

def calculate_orbital_elements(m1, m2, pos1, pos2, vel1, vel2, m3, pos3, vel3):
    print("   Calculating semimajor axis and eccentricity evolution of the giant's orbit")

    m12 = m1+m2
    rel_position = (m1 * pos1 + m2 * pos2)/m12 - pos3
    rel_velocity = (m1 * vel1 + m2 * vel2)/m12 - vel3
    mtot = m12 + m3
    separation = rel_position.lengths()
    speed_squared = rel_velocity.lengths_squared()
    
    # Now calculate the important quantities:
    semimajor_axis = (constants.G * mtot * separation / 
        (2 * constants.G * mtot - separation * speed_squared)).as_quantity_in(units.AU)
    eccentricity = numpy.sqrt(1.0 - (rel_position.cross(rel_velocity)**2).sum(axis=1) / 
        (constants.G * mtot * semimajor_axis))
    return semimajor_axis, eccentricity

def make_movie():
    print("   Creating movie from snapshots")
    try:
        subprocess.call(['mencoder', "mf://hydro_triple_small*.png", '-ovc', 'lavc', 
            '-o', '../hydro_triple_small.avi', '-msglevel', 'all=1'], cwd="./plots")
        subprocess.call(['mencoder', "mf://hydro_triple_large*.png", '-ovc', 'lavc', 
            '-o', '../hydro_triple_large.avi', '-msglevel', 'all=1'], cwd="./plots")
    except Exception as exc:
        print("   Failed to create movie, error was:", str(exc))

def continue_evolution(sph_code, dynamics_code, t_end, n_steps, 
        relaxed_giant_output_base_name, do_energy_evolution_plot):
    print("Loading snapshots...", end=' ')
    files = os.listdir("snapshots")
    files.sort()
    files = files[-4:]
    print(files)
    binary = read_set_from_file(os.path.join("snapshots", files[0]), format='amuse')
    gd_particles = read_set_from_file(os.path.join("snapshots", files[1]), format='amuse')
    sph_particles = read_set_from_file(os.path.join("snapshots", files[2]), format='amuse')
    
    snapshotfile = os.path.join("..", "giant_models", relaxed_giant_output_base_name + "_core.amuse")
    core_particle = read_set_from_file(snapshotfile, format='amuse')
    
    giant_model = StellarModelInSPH(
        gas_particles = sph_particles, 
        core_particle = gd_particles[0], 
        core_radius = core_particle.radius)
    
    view_on_giant = Particle()
    view_on_giant.position = [0]*3 | units.m
    view_on_giant.velocity = [0]*3 | units.m / units.s
    
    print("\nSetting up {0} to simulate inner binary system".format(dynamics_code.__name__))
    binary_system = prepare_binary_system(dynamics_code, binary)
    
    print("\nSetting up {0} to simulate giant in SPH".format(sph_code.__name__))
    giant_system = prepare_giant_system(sph_code, giant_model, view_on_giant, t_end, n_steps)
    
    print("\nEvolving with bridge between", sph_code.__name__, "and", dynamics_code.__name__)
    evolve_coupled_system(binary_system, giant_system, t_end, n_steps, do_energy_evolution_plot, 
        previous_data = os.path.join("snapshots", files[3]))
    print("Done")

def energy_evolution_plot(time, kinetic, potential, thermal, figname = "energy_evolution.png"):
    time.prepend(0.0 | units.day)
    pyplot.figure(figsize = (5, 5))
    plot(time, kinetic, label='K')
    plot(time, potential, label='U')
    plot(time, thermal, label='Q')
    plot(time, kinetic + potential + thermal, label='E')
    xlabel('Time')
    ylabel('Energy')
    pyplot.legend(prop={'size':"x-small"}, loc=4)
    pyplot.savefig(figname)
    pyplot.close()

def orbit_ecc_plot(eccentricity_in,eccentricity_out,time):
    figure = pyplot.figure(figsize = (10, 6), dpi = 100)
    subplot = figure.add_subplot(2, 1, 1)
    plot(time,eccentricity_in)
    xlabel('t')
    ylabel('e$_\mathrm{binary}$')
    
    subplot = figure.add_subplot(2, 1, 2)
    plot(time,eccentricity_out )  
    xlabel('t')
    ylabel('e$_\mathrm{giant}$')
    pyplot.minorticks_on()
    pyplot.savefig("eccentricity_evolution.png")
    pyplot.close()


def orbit_parameters_plot(semi_major_in,semi_major_out, time, par_symbol="a", par_name="semimajor_axis"):
    figure = pyplot.figure(figsize = (10, 6), dpi = 100)
    subplot = figure.add_subplot(2, 1, 1)
    plot(time,semi_major_in )
    xlabel('t')
    ylabel('$'+par_symbol+'_\mathrm{binary}$')
    
    subplot = figure.add_subplot(2, 1, 2)
    plot(time,semi_major_out )
    xlabel('t')
    ylabel('$'+par_symbol+'_\mathrm{giant}$')
    pyplot.minorticks_on()
    pyplot.savefig(par_name+"_evolution.png")
    pyplot.close()
    
def evolve_coupled_system(binary_system, giant_system, giant_model,inner_binary, hydro_channels,gravity_channels, \
                          t_end, n_steps, do_energy_evolution_plot, previous_data=None):
    
    directsum = CalculateFieldForParticles(particles=giant_system.particles, gravity_constant=constants.G)  # more info ./src/amuse/couple/bridge.py
    directsum.smoothing_length_squared = giant_system.parameters.gas_epsilon**2
    coupled_system = Bridge(timestep=(t_end / (2 * n_steps)), verbose=False, use_threading=True)
    coupled_system.add_system(binary_system, (directsum,), False)
    coupled_system.add_system(giant_system, (binary_system,), False)
    
    times = (t_end * list(range(1, n_steps+1)) / n_steps).as_quantity_in(units.day)
    
    if previous_data:
        with open(previous_data, 'rb') as file:
            (all_times, potential_energies, kinetic_energies, thermal_energies, 
                giant_center_of_mass, ms1_position, ms2_position, 
                giant_center_of_mass_velocity, ms1_velocity, ms2_velocity) = pickle.load(file)
        all_times.extend(times + all_times[-1])
    else:
        all_times = times
        if do_energy_evolution_plot:
            potential_energies = coupled_system.particles.potential_energy().as_vector_with_length(1).as_quantity_in(units.erg)
            kinetic_energies = coupled_system.particles.kinetic_energy().as_vector_with_length(1).as_quantity_in(units.erg)
            thermal_energies = coupled_system.gas_particles.thermal_energy().as_vector_with_length(1).as_quantity_in(units.erg)
        else:
            potential_energies = kinetic_energies = thermal_energies = None  
        
    #Giant
    giant_center_of_mass = [] | units.au
    ms1_position = [] | units.au
    ms2_position = [] | units.au
    giant_center_of_mass_velocity = [] | units.km / units.s
    ms1_velocity = [] | units.km / units.s
    ms2_velocity = [] | units.km / units.s

    # Create sink particles based on the inner binary
    sink_cand = inner_binary.copy()
    sink_particles = new_sink_particles(sink_cand)
    # Calculate the accretion radii of the sinks
    sink_particles = update_sinks_radii(sink_particles)
    gravity_channels["sinks_to_code"] = sink_particles.new_channel_to(binary_system.particles)
    gravity_channels["code_to_sinks"] = binary_system.particles.new_channel_to(sink_particles) 

    # Allow the sink particles to accrete gas particles  
    sink_particles.accrete(giant_model.gas_particles) 
    # update the code gas particles based on the local particles
    giant_model.gas_particles.synchronize_to(giant_system.gas_particles)
    # update the inner binary IN the code based on the sink particles
    gravity_channels["sinks_to_code"].copy()
    # update the local inner binary based on the inner binary IN the code
    gravity_channels["code_to_local"].copy()

    i_offset = len(giant_center_of_mass)

    giant_total_mass = giant_system.particles.total_mass()
    ms1_mass = binary_system.particles[0].mass
    ms2_mass = binary_system.particles[1].mass
    
    for i_step, time in enumerate(times):   
        coupled_system.evolve_model(time)
        
        print("   Evolved to:", time, end=' ')
        #update the gas particles FROM the code
        hydro_channels["code_to_local_gas"].copy()
        #update the core particle FROM the code
        hydro_channels["code_to_local_core"].copy()
        #update the sink particles [vx ,vy,vz,x ,y,z] FROM the code
        gravity_channels["code_to_sinks"].copy()

        # acrrete gas particles if possible
        tot_sinks = sink_particles.mass.sum()
        sink_particles.accrete(giant_model.gas_particles)
        dM_sinks = sink_particles.mass.sum() - tot_sinks

        if dM_sinks>0|units.MSun:
            print("\nStars accreted dM=", dM_sinks.in_(units.MSun))
            # synchronize the code gas particles with the local gas particles in the code and remove the accreted ones
            giant_model.gas_particles.synchronize_to(giant_system.gas_particles)
            # update the accretion radius of the sinks based on the new mass and positions
            sink_particles = update_sinks_radii(sink_particles)
            #update the inner particles [mass,accretion_radius] in the code
            gravity_channels["sinks_to_code"].copy()
            
            print(len(giant_system.gas_particles),len(giant_model.gas_particles),len(coupled_system.gas_particles))
            print('/n',len(coupled_system.particles))
            
        #update the inner binary particles [mass,radius,vx ,vy,vz,x ,y,z] FROM the code
        gravity_channels["code_to_local"].copy()
        
        giant_total_mass = giant_system.particles.total_mass()
        ms1_mass = binary_system.particles[0].mass
        ms2_mass = binary_system.particles[1].mass
              
        if do_energy_evolution_plot:
            potential_energies.append(coupled_system.particles.potential_energy())
            kinetic_energies.append(coupled_system.particles.kinetic_energy())
            thermal_energies.append(coupled_system.gas_particles.thermal_energy())
        
        giant_center_of_mass.append(giant_system.particles.center_of_mass())
        ms1_position.append(binary_system.particles[0].position)
        ms2_position.append(binary_system.particles[1].position)
        giant_center_of_mass_velocity.append(giant_system.particles.center_of_mass_velocity())
        ms1_velocity.append(binary_system.particles[0].velocity)
        ms2_velocity.append(binary_system.particles[1].velocity)

        a_giant, e_giant = calculate_orbital_elements(ms1_mass, ms2_mass,
                                                      ms1_position, ms2_position,
                                                      ms1_velocity, ms2_velocity,
                                                      giant_total_mass,
                                                      giant_center_of_mass,
                                                      giant_center_of_mass_velocity)
        
        print("Outer Orbit:", time.in_(units.day),  a_giant[-1].in_(units.AU), e_giant[-1], ms1_mass.in_(units.MSun), ms2_mass.in_(units.MSun), giant_total_mass.in_(units.MSun))
        if i_step % 10 == 9:
            snapshotfile = os.path.join("snapshots", "hydro_triple_{0:=04}_gas.amuse".format(i_step + i_offset))
            write_set_to_file(giant_system.gas_particles, snapshotfile, format='amuse')
            snapshotfile = os.path.join("snapshots", "hydro_triple_{0:=04}_core.amuse".format(i_step + i_offset))
            write_set_to_file(giant_system.dm_particles, snapshotfile, format='amuse')
            snapshotfile = os.path.join("snapshots", "hydro_triple_{0:=04}_binary.amuse".format(i_step + i_offset))
            write_set_to_file(binary_system.particles, snapshotfile, format='amuse')

            datafile = os.path.join("snapshots", "hydro_triple_{0:=04}_info.amuse".format(i_step + i_offset))
            with open(datafile, 'wb') as outfile:
                pickle.dump((all_times[:len(giant_center_of_mass)], potential_energies, 
                    kinetic_energies, thermal_energies, giant_center_of_mass, 
                    ms1_position, ms2_position, giant_center_of_mass_velocity,
                    ms1_velocity, ms2_velocity), outfile)

        figname1 = os.path.join("plots", "hydro_triple_small{0:=04}.png".format(i_step + i_offset))
        figname2 = os.path.join("plots", "hydro_triple_large{0:=04}.png".format(i_step + i_offset))
        print("  -   Hydroplots are saved to: ", figname1, "and", figname2)
        for plot_range, plot_name in [(8|units.AU, figname1), (40|units.AU, figname2)]:
            if HAS_PYNBODY:
                pynbody_column_density_plot(coupled_system.gas_particles, width=plot_range, vmin=26, vmax=32)
                scatter(coupled_system.dm_particles.x, coupled_system.dm_particles.y, c="w")
            else:
                pyplot.figure(figsize = [16, 16])
                sph_particles_plot(coupled_system.gas_particles, gd_particles=coupled_system.dm_particles,
                                   view=plot_range*[-0.5, 0.5, -0.5, 0.5])
            pyplot.savefig(plot_name)
            pyplot.close()


    coupled_system.stop()

    #make_movie()

    if do_energy_evolution_plot:
        energy_evolution_plot(all_times[:len(kinetic_energies)-1], kinetic_energies, 
            potential_energies, thermal_energies)

    print("   Calculating semimajor axis and eccentricity evolution for inner binary")
    # Some temporary variables to calculate semimajor_axis and eccentricity evolution
    total_mass = ms1_mass + ms2_mass
    rel_position = ms1_position - ms2_position
    rel_velocity = ms1_velocity - ms2_velocity
    separation_in = rel_position.lengths()
    speed_squared_in = rel_velocity.lengths_squared()

    # Now calculate the important quantities:
    semimajor_axis_binary = (constants.G * total_mass * separation_in / 
        (2 * constants.G * total_mass - separation_in * speed_squared_in)).as_quantity_in(units.AU)
    eccentricity_binary = numpy.sqrt(1.0 - (rel_position.cross(rel_velocity)**2).sum(axis=1) / 
        (constants.G * total_mass * semimajor_axis_binary))

    print("   Calculating semimajor axis and eccentricity evolution of the giant's orbit")
    # Some temporary variables to calculate semimajor_axis and eccentricity evolution
    rel_position = ((ms1_mass * ms1_position + ms2_mass * ms2_position)/total_mass - 
        giant_center_of_mass)
    rel_velocity = ((ms1_mass * ms1_velocity + ms2_mass * ms2_velocity)/total_mass - 
        giant_center_of_mass_velocity)
    total_mass += giant_total_mass
    separation = rel_position.lengths()
    speed_squared = rel_velocity.lengths_squared()

    # Now calculate the important quantities:
    semimajor_axis_giant = (constants.G * total_mass * separation / 
        (2 * constants.G * total_mass - separation * speed_squared)).as_quantity_in(units.AU)
    eccentricity_giant = numpy.sqrt(1.0 - (rel_position.cross(rel_velocity)**2).sum(axis=1) / 
        (constants.G * total_mass * semimajor_axis_giant))

    orbit_parameters_plot(semimajor_axis_binary, semimajor_axis_giant, all_times[:len(semimajor_axis_binary)])
    orbit_ecc_plot(eccentricity_binary, eccentricity_giant, all_times[:len(eccentricity_binary)])

    orbit_parameters_plot(separation_in.as_quantity_in(units.AU), 
        separation.as_quantity_in(units.AU), 
        all_times[:len(eccentricity_binary)], 
        par_symbol="r", par_name="separation")
    orbit_parameters_plot(speed_squared_in.as_quantity_in(units.km**2 / units.s**2), 
        speed_squared.as_quantity_in(units.km**2 / units.s**2), 
        all_times[:len(eccentricity_binary)], 
        par_symbol="v^2", par_name="speed_squared")


# Main

## Stellar evolution set up

In [28]:
triple, view_on_giant = set_up_initial_conditions()
radius_factor= 1.0
stop_radius = radius_factor * estimate_roche_radius(triple, view_on_giant)
print("Tertiary's Roche lobe radius is {:.2f} RSun or {:.2f} AU".format(stop_radius.value_in(units.RSun), \
                                                                     stop_radius.value_in(units.au)))

triple_set_up_info(triple, view_on_giant)

   Initializing inner binary
   Orbital period inner binary: 1.1 day
   Initializing outer star
   Initializing outer star
   Orbital period outer binary: 52.04
Tertiary's Roche lobe radius is 71.82 RSun or 0.33 AU

 Inner orbit:
Orbital Separation of inner binary stars 0.05 AU or 10.38 RSun
Semi-major axis1 0.02 AU or 4.94 RSun
Semi-major axis1 0.03 AU or 5.44 RSun

 Outer orbit:

Binary system semi-major axis is 0.47 AU or 100.86 RSun
Giant's semi-major is axis 0.36 AU or 78.17 RSun
Giant's distance from binary's COM is 0.83 AU or 179.03 RSun


In [6]:
#se_code.default_path_to_inlist
#se_code.parameters.herwig_convective_overshoot_parameter = 0.0
#se_code.parameters.semi_convection_efficiency = 0.0

## Code  set up

In [7]:
se_code = MESA(version='2208')
print(se_code.parameters)

se_stars, se_code_instance , stellar_evolution_output = evolve_stars(triple, view_on_giant, se_code, radius_factor)
triple[2].radius = se_stars[0].radius
triple[0].radius = se_stars[1].radius
triple[1].radius = se_stars[2].radius
#print("\nStellar evolution done:\n", se_stars)

Invalid MIT-MAGIC-COOKIE-1 key

AGB_wind_scheme: 1
RGB_wind_scheme: 1
blocker_wind_efficiency: 0.1
de_jager_wind_efficiency: 0.8
dutch_wind_efficiency: 0.8
herwig_convective_overshoot_parameter: 0.0
max_age_stop_condition: 1e+36 yr
max_iter_stop_condition: -1111
metallicity: 0.02
min_timestep_stop_condition: 1e-06 s
mixing_length_ratio: 2.0
reimers_wind_efficiency: 0.5
semi_convection_efficiency: 0.0
stabilize_new_stellar_model_flag: True

Roche Lobe radius of the tertiary =0.33 au
15.9999998484 MSun 1.55263526304e-09 MSun / yr 9.765625e-05 Myr 5.1092438154 RSun Main Sequence star
15.9999996686 MSun 1.53396583261e-09 MSun / yr 0.00021484375 Myr 5.11463387173 RSun Main Sequence star
15.999999454 MSun 1.52654198759e-09 MSun / yr 0.00035546875 Myr 5.12394584566 RSun Main Sequence star
15.9999991984 MSun 1.51489521119e-09 MSun / yr 0.00052421875 Myr 5.13780462658 RSun Main Sequence star
15.9999988946 MSun 1.50009200193e-09 MSun / yr 0.00072671875 Myr 5.15486982099 RSun Main Sequence star
15.9999985752 MSun 1.48313852551e

15.9968548551 MSun 1.82580719491e-09 MSun / yr 1.89372441627 Myr 5.64536029906 RSun Main Sequence star
15.9967700358 MSun 1.83472984012e-09 MSun / yr 1.93995897322 Myr 5.65817058332 RSun Main Sequence star
15.9966844969 MSun 1.84367571064e-09 MSun / yr 1.98635957789 Myr 5.67104482095 RSun Main Sequence star
15.9965982585 MSun 1.85271376588e-09 MSun / yr 2.03291139386 Myr 5.68400910275 RSun Main Sequence star
15.996511607 MSun 1.86186164484e-09 MSun / yr 2.07945639268 Myr 5.69710938851 RSun Main Sequence star
15.9964247595 MSun 1.8710966606e-09 MSun / yr 2.1258764182 Myr 5.71019229579 RSun Main Sequence star
15.9963372998 MSun 1.88037323708e-09 MSun / yr 2.1723930377 Myr 5.72331176064 RSun Main Sequence star
15.9962485572 MSun 1.88970601152e-09 MSun / yr 2.21935889687 Myr 5.73657364821 RSun Main Sequence star
15.9961592231 MSun 1.89918239591e-09 MSun / yr 2.26640187903 Myr 5.74689465255 RSun Main Sequence star
15.996068163 MSun 1.90886087672e-09 MSun / yr 2.31411062227 Myr 5.76025556751

15.9884361808 MSun 2.86031110289e-09 MSun / yr 5.58549672586 Myr 6.9917659067 RSun Main Sequence star
15.9883054643 MSun 2.87978273956e-09 MSun / yr 5.63089245403 Myr 7.01612839658 RSun Main Sequence star
15.9881754875 MSun 2.89950277279e-09 MSun / yr 5.67572429287 Myr 7.04048973014 RSun Main Sequence star
15.9880452041 MSun 2.9192590036e-09 MSun / yr 5.72035778954 Myr 7.06530926314 RSun Main Sequence star
15.987914105 MSun 2.93932223903e-09 MSun / yr 5.76496413284 Myr 7.09027655336 RSun Main Sequence star
15.9877826689 MSun 2.9595391198e-09 MSun / yr 5.80937968447 Myr 7.11546904639 RSun Main Sequence star
15.9876500746 MSun 2.98002036894e-09 MSun / yr 5.85387863193 Myr 7.14103233539 RSun Main Sequence star
15.9875160277 MSun 3.00073644252e-09 MSun / yr 5.89855452383 Myr 7.16722434579 RSun Main Sequence star
15.9873806401 MSun 3.02195494492e-09 MSun / yr 5.9433604414 Myr 7.19367983072 RSun Main Sequence star
15.9872445971 MSun 3.04341570525e-09 MSun / yr 5.98806573984 Myr 7.22054376071

15.9746734337 MSun 5.79423568022e-09 MSun / yr 9.0246870797 Myr 10.5781813911 RSun Main Sequence star
15.9744738832 MSun 5.85337130729e-09 MSun / yr 9.05878210613 Myr 10.6480389244 RSun Main Sequence star
15.9742732658 MSun 5.91323602834e-09 MSun / yr 9.09271240379 Myr 10.7182705851 RSun Main Sequence star
15.9740712102 MSun 5.97370649123e-09 MSun / yr 9.12654001091 Myr 10.7890449642 RSun Main Sequence star
15.9738671179 MSun 6.03490175055e-09 MSun / yr 9.16036211782 Myr 10.8603331764 RSun Main Sequence star
15.9736607582 MSun 6.09686966565e-09 MSun / yr 9.19421239885 Myr 10.9318417247 RSun Main Sequence star
15.9734518153 MSun 6.15957816435e-09 MSun / yr 9.22813748915 Myr 11.0041333312 RSun Main Sequence star
15.97323998 MSun 6.22331924355e-09 MSun / yr 9.26217991095 Myr 11.0766712149 RSun Main Sequence star
15.97302491 MSun 6.28796882635e-09 MSun / yr 9.29638680647 Myr 11.1494612944 RSun Main Sequence star
15.9728072177 MSun 6.35346895251e-09 MSun / yr 9.33065383747 Myr 11.221731527 

15.969351953 MSun 1.07768989524e-08 MSun / yr 9.8376114417 Myr 15.400213258 RSun First Giant Branch
15.969349684 MSun 1.08816184152e-08 MSun / yr 9.83781998278 Myr 15.5295115312 RSun First Giant Branch
15.9693473936 MSun 1.09871429263e-08 MSun / yr 9.83802846283 Myr 15.6599358755 RSun First Giant Branch
15.969345082 MSun 1.10933592886e-08 MSun / yr 9.83823686602 Myr 15.7914705099 RSun First Giant Branch
15.9693427491 MSun 1.12002700014e-08 MSun / yr 9.8384451724 Myr 15.924405705 RSun First Giant Branch
15.9693403887 MSun 1.13071487113e-08 MSun / yr 9.83865394597 Myr 16.0588961691 RSun First Giant Branch
15.9693380068 MSun 1.14148342167e-08 MSun / yr 9.83886264114 Myr 16.1945854123 RSun First Giant Branch
15.9693356115 MSun 1.15229677224e-08 MSun / yr 9.83907053238 Myr 16.3311778373 RSun First Giant Branch
15.9693332016 MSun 1.16313128093e-08 MSun / yr 9.83927774412 Myr 16.4687396058 RSun First Giant Branch
15.9693307765 MSun 1.17399272552e-08 MSun / yr 9.83948433148 Myr 16.6073066101 R

15.9691254111 MSun 2.22300233709e-08 MSun / yr 9.85219458676 Myr 32.1212093739 RSun First Giant Branch
15.9691222031 MSun 2.24353912009e-08 MSun / yr 9.85233759075 Myr 32.4574143079 RSun First Giant Branch
15.9691189809 MSun 2.264316489e-08 MSun / yr 9.85247990667 Myr 32.7988476384 RSun First Giant Branch
15.9691157482 MSun 2.28534904953e-08 MSun / yr 9.85262137484 Myr 33.1453215214 RSun First Giant Branch
15.9691125098 MSun 2.30663151952e-08 MSun / yr 9.8527617842 Myr 33.4959439798 RSun First Giant Branch
15.9691092567 MSun 2.3281128205e-08 MSun / yr 9.85290153008 Myr 33.851782822 RSun First Giant Branch
15.9691059883 MSun 2.34984983651e-08 MSun / yr 9.85304063291 Myr 34.2139601755 RSun First Giant Branch
15.9691027084 MSun 2.37193234546e-08 MSun / yr 9.85317892695 Myr 34.5811889892 RSun First Giant Branch
15.9690999686 MSun 2.39427960321e-08 MSun / yr 9.85329337021 Myr 34.8911897564 RSun First Giant Branch
15.9690975464 MSun 2.41311583715e-08 MSun / yr 9.8533937586 Myr 35.1674679418 

In [8]:
# for stars in se_stars:
#     print(stars.age.in_(units.Myr), stars.mass.in_(units.MSun), stars.radius.in_(units.RSun),
#           stars.wind.in_(units.MSun / units.yr))
# print(r'Tertiary mass loss due to winds {:.3f} MSun'.format(16  - se_stars[0].mass.value_in(units.MSun)))

In [9]:
print('Physical M_core {:.2f} MSun'.format(se_stars[0].core_mass.value_in(units.MSun)))

Physical M_core 3.20 MSun


# SPH Code

In [10]:
number_of_sph_particles = 1000
# Stop stellar evolution when giant's radius is (radius_factor * Roche lobe radius)
radius_factor = 1.0
relaxed_giant_output_base_name = "relaxed_giant_" + str(number_of_sph_particles) + "_" + str(radius_factor)
t_end = 100.0 | units.day
n_steps = 200
view_on_se_giant = view_on_giant.as_set().get_intersecting_subset_in(se_stars)[0]
giant_model = convert_giant_to_sph(view_on_se_giant, number_of_sph_particles)
se_code_instance.stop()

# Giant's info before relaxation

In [11]:
total_radius = max(giant_model.gas_particles.position.lengths_squared()).sqrt()
total_radius.in_(units.RSun)

quantity<51.7987644343 RSun>

In [12]:
tot_mass = giant_model.gas_particles.total_mass() + giant_model.core_particle.mass  
tot_radius = max(giant_model.gas_particles.position.lengths_squared()).sqrt()

dynamical_timescale = (tot_radius**3 / (2 * constants.G * tot_mass)).sqrt().as_quantity_in(units.day) 

print("Tertiary's core mass is {:.2f} MSun, while envelope's mass is {:.2f} MSun" \
      .format(giant_model.core_particle.mass.value_in(units.MSun), \
              giant_model.gas_particles.total_mass().value_in(units.MSun)))

print("The dynamical scale of the Giant is {:0.2f} days".format(dynamical_timescale.value_in(units.day)))

print("Tertiary's Roche lobe radius is {:.2f} RSun".format(stop_radius.value_in(units.RSun)))

a_giant = (view_on_giant.position - triple.center_of_mass()).length()
print("Semi-major axis of tertiary's orbit  is {:.2f} au".format(a_giant.value_in(units.au)))
a_giant = (giant_model.gas_particles.center_of_mass() - triple.center_of_mass()).length()
print("Semi-major axis of tertiary's orbit  is {:.2f} au".format(a_giant.value_in(units.au)))
a_giant = (giant_model.core_particle.position - triple.center_of_mass()).length()
print("Semi-major axis of tertiary's orbit  is {:.2f} au".format(a_giant.value_in(units.au)))

Tertiary's core mass is 7.01 MSun, while envelope's mass is 8.96 MSun
The dynamical scale of the Giant is 1.22 days
Tertiary's Roche lobe radius is 71.82 RSun
Semi-major axis of tertiary's orbit  is 0.36 au
Semi-major axis of tertiary's orbit  is 0.00 au
Semi-major axis of tertiary's orbit  is 0.00 au


In [13]:
sph_code = Gadget2
print("Relaxing giant with", sph_code.__name__)
mult_factor = 5.0
relax_in_isolation(giant_model, sph_code, relaxed_giant_output_base_name,mult_factor)

Relaxing giant with Gadget2
Relaxing for 6.08 (5 * dynamical timescale)
   Relaxed for: 0.0607741856257 day
   Relaxed for: 0.121548371251 day
   Relaxed for: 0.182322556877 day
   Relaxed for: 0.243096742503 day
   Relaxed for: 0.303870928128 day
   Relaxed for: 0.364645113754 day
   Relaxed for: 0.42541929938 day
   Relaxed for: 0.486193485005 day
   Relaxed for: 0.546967670631 day
   Relaxed for: 0.607741856257 day
   Relaxed for: 0.668516041883 day
   Relaxed for: 0.729290227508 day
   Relaxed for: 0.790064413134 day
   Relaxed for: 0.85083859876 day
   Relaxed for: 0.911612784385 day
   Relaxed for: 0.972386970011 day
   Relaxed for: 1.03316115564 day
   Relaxed for: 1.09393534126 day
   Relaxed for: 1.15470952689 day
   Relaxed for: 1.21548371251 day
   Relaxed for: 1.27625789814 day
   Relaxed for: 1.33703208377 day
   Relaxed for: 1.39780626939 day
   Relaxed for: 1.45858045502 day
   Relaxed for: 1.51935464064 day
   Relaxed for: 1.58012882627 day
   Relaxed for: 1.64090301189

# Giant's info before relaxation

In [14]:
total_radius = max(giant_model.gas_particles.position.lengths_squared()).sqrt()
total_radius.in_(units.RSun)

quantity<48.7015369714 RSun>

In [15]:
tot_mass = giant_model.gas_particles.total_mass() + giant_model.core_particle.mass  
tot_radius = max(giant_model.gas_particles.position.lengths_squared()).sqrt()

dynamical_timescale = (tot_radius**3 / (2 * constants.G * tot_mass)).sqrt().as_quantity_in(units.day) 

print("Tertiary's core mass is {:.2f} MSun, while envelope's mass is {:.2f} MSun" \
      .format(giant_model.core_particle.mass.value_in(units.MSun), \
              giant_model.gas_particles.total_mass().value_in(units.MSun)))

print("The dynamical scale of the Giant is {:0.2f} days".format(dynamical_timescale.value_in(units.day)))

print("Tertiary's Roche lobe radius is {:.2f} RSun".format(stop_radius.value_in(units.RSun)))

a_giant = (view_on_giant.position - triple.center_of_mass()).length()
print("Semi-major axis of tertiary's orbit  is {:.2f} au".format(a_giant.value_in(units.au)))
a_giant = (giant_model.gas_particles.center_of_mass() - triple.center_of_mass()).length()
print("Semi-major axis of tertiary's orbit  is {:.2f} au".format(a_giant.value_in(units.au)))
a_giant = (giant_model.core_particle.position - triple.center_of_mass()).length()
print("Semi-major axis of tertiary's orbit  is {:.2f} au".format(a_giant.value_in(units.au)))

Tertiary's core mass is 7.01 MSun, while envelope's mass is 8.96 MSun
The dynamical scale of the Giant is 1.11 days
Tertiary's Roche lobe radius is 71.82 RSun
Semi-major axis of tertiary's orbit  is 0.36 au
Semi-major axis of tertiary's orbit  is 0.06 au
Semi-major axis of tertiary's orbit  is 0.06 au


In [16]:
dynamics_code = Huayno
inner_binary = triple - view_on_giant
print("\nSetting up {0} to simulate inner binary system".format(dynamics_code.__name__))
binary_system = prepare_binary_system(dynamics_code, inner_binary)

# Create a channel between the inner_binary particles (python) and the binary_system.particles (Huayno)
gravity_channels = {}
gravity_channels["local_to_code"] = inner_binary.new_channel_to(binary_system.particles)
gravity_channels["code_to_local"] = binary_system.particles.new_channel_to(inner_binary)


Setting up Huayno to simulate inner binary system


In [18]:
print("\nSetting up {0} to simulate giant in SPH".format(sph_code.__name__))
giant_system = prepare_giant_system(sph_code, giant_model, view_on_giant, t_end, n_steps)
# Create a channel between the giant_model particles (python) and the giant_system.particles (Gadget,Huayno)
hydro_channels = {}
hydro_channels["local_to_code"] = giant_model.gas_particles.new_channel_to(giant_system.gas_particles)
hydro_channels["code_to_local_gas"] = giant_system.gas_particles.new_channel_to(giant_model.gas_particles)
hydro_channels["code_to_local_core"] = giant_system.dm_particles.new_channel_to(giant_model.core_particle.as_set())



Setting up Gadget2 to simulate giant in SPH


In [ ]:
do_energy_evolution_plot =True
evolve_coupled_system(binary_system, giant_system, giant_model,inner_binary, hydro_channels,gravity_channels, \
                    t_end, n_steps, do_energy_evolution_plot, previous_data=None)